In [ ]:
! pip install -q simpletransformers

     |████████████████████████████████| 248 kB 4.1 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 1.7 MB 76.8 MB/s 
     |████████████████████████████████| 3.5 MB 67.7 MB/s 
     |████████████████████████████████| 6.8 MB 81.9 MB/s 
     |████████████████████████████████| 311 kB 99.2 MB/s 
     |████████████████████████████████| 9.7 MB 70.0 MB/s 
     |████████████████████████████████| 1.2 MB 89.8 MB/s 
     |████████████████████████████████| 596 kB 90.9 MB/s 
     |████████████████████████████████| 895 kB 94.3 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 180 kB 102.6 MB/s 
     |████████████████████████████████| 144 kB 104.9 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 MB 80.3 MB/s 
     |████████████████████████████████| 243 kB 102.0 MB/s 
     |████████████████████████████████| 133 kB 109.2 MB/s 
     |███████████

In [ ]:
!pip install wandb

In [ ]:
!wandb login 28d2de59b2adb6064f3125b6c41e3914823d3cbd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
! pip install -q datasets transformers[sentencepiece] simpletransformers

In [ ]:
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz

--2022-02-17 12:32:25--  https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kobkrit/datasets/main/AIFORTHAI-LST20Corpus.tar.gz [following]
--2022-02-17 12:32:26--  https://raw.githubusercontent.com/kobkrit/datasets/main/AIFORTHAI-LST20Corpus.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13551362 (13M) [application/octet-stream]
Saving to: ‘AIFORTHAI-LST20Corpus.tar.gz’

AIFORTHAI-LST20Corp 100%[===================>]  12.92M  --.-KB/s    in 0.1s    

2022-02-17 12:32:27 (101 MB/s) - ‘AIFORTHAI-LST20Corpus.tar.gz’ save

In [ ]:
! tar -xvzf AIFORTHAI-LST20Corpus.tar.gz

In [ ]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="/content/LST20_Corpus")

Using custom data configuration default-97621787f5b60e1b


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset lst20 downloaded and prepared to /root/.cache/huggingface/datasets/lst20/default-97621787f5b60e1b/0.0.0/e1b2a921fb011578ab43ddbbf789f3c500d62cb2df8ae4ed4b60bae8e4c0d3ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])

In [ ]:
train_df

,id,fname,tokens,pos_tags,ner_tags,clause_tags
0,0,T00126.txt,"[สุรยุทธ์, ยัน, ปฏิเสธ, ลงนาม, _, MOU, _, กับ,...","[0, 1, 1, 1, 2, 0, 2, 4, 0, 12, 1, 0]","[8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]"
1,1,T00126.txt,"[1, _, กันยายน, _, 2550, _, 12:21, _, น.]","[8, 2, 0, 2, 8, 2, 8, 2, 10]","[3, 13, 13, 13, 23, 0, 3, 13, 23]","[1, 2, 2, 2, 2, 2, 2, 2, 3]"
2,2,T00126.txt,"[นายก, ฯ, _, ย้ำ, ไม่, ลงนาม, ใน, _, MOU, _, ร...","[0, 2, 2, 1, 12, 1, 4, 2, 0, 2, 4, 0, 3, 5, 6,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, ..."
3,3,T00126.txt,"[ยัน, ไทย, ไม่, ได้, ล้มเหลว, ถึง, ขั้น, นั้น,...","[1, 0, 12, 5, 1, 1, 0, 9, 1, 7, 1, 0, 9, 12, 1...","[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,4,T00126.txt,"[พร้อม, หนุน, กกต., จัด, เลือกตั้ง, ให้, บริสุ...","[3, 1, 0, 1, 1, 3, 1, 1]","[0, 0, 7, 0, 0, 0, 0, 0]","[1, 2, 2, 2, 3, 1, 2, 3]"
...,...,...,...,...,...,...
63305,6,T13136.txt,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, หาก, การ, ยื่น...","[0, 0, 2, 1, 3, 2, 3, 7, 1, 0, 12, 1, 3, 5, 1,...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 2, 3, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63306,7,T13136.txt,"[เมื่อ, ถาม, ว่า, _, ใน, ทาง, การเมือง, _, หาก...","[4, 1, 3, 2, 4, 0, 0, 2, 3, 0, 3, 1, 0, 6, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63307,8,T13136.txt,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, เรื่อง, กฎหมาย...","[0, 0, 2, 1, 3, 2, 0, 0, 5, 3, 1, 2, 7, 1, 1, ...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63308,9,T13136.txt,"[หาก, เป็น, การ, ขอ, ถวาย, ฎีกา, ตาม, โบราณ, ร...","[3, 1, 7, 1, 1, 0, 4, 0, 0, 2, 5, 1, 0, 3, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, ..."


In [ ]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

In [ ]:
def convert_to_simple_transformer_format(df, field_name, tags):
  sentence_id = []
  words = []
  labels = []

  #Limit at 1000 rows for speed.
  for (idx, r) in df.iterrows():
    # print(idx)
    for (i, t) in enumerate(r['tokens']):
      # print(i,t)
      sentence_id.append(idx)
      words.append(t)
      labels.append(tags[r[field_name][i]])
    
  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels}
  )

In [ ]:
train_ = convert_to_simple_transformer_format(train_df, "pos_tags", _POS_TAGS)
validation_ = convert_to_simple_transformer_format(validation_df, "pos_tags", _POS_TAGS)
test_ = convert_to_simple_transformer_format(test_df, "pos_tags", _POS_TAGS)

In [ ]:
train_

,sentence_id,words,labels
0,0,สุรยุทธ์,NN
1,0,ยัน,VV
2,0,ปฏิเสธ,VV
3,0,ลงนาม,VV
4,0,_,PU
...,...,...,...
2714721,63309,การ,FX
2714722,63309,ถอน,VV
2714723,63309,รายชื่อ,NN
2714724,63309,ด้วย,PS


In [ ]:
import logging
import torch
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
######### Tune Hyper parameter ########

# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 8
model_args.learning_rate = 1e-5
model_args.polynomial_decay_schedule_lr_end = 1e-9
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = True
model_args.use_early_stopping = False
model_args.weight_decay = 1e-2
model_args.num_train_epochs = 30
model_args.max_seq_length = 128
model_args.wandb_project = 'nlp-POS-tag-hackatron'
model_args.best_model_dir = '/content/drive/MyDrive/POSTAG/BERT_model/Roberta/'

model = NERModel(
    "roberta",'/content/drive/MyDrive/POSTAG/BERT_model/Roberta', args=model_args, use_cuda=torch.cuda.is_available(), labels=_POS_TAGS
)

# Train the model
model.train_model(train_, eval_data=validation_)

In [ ]:
model = NERModel(
    "roberta",'/content/drive/MyDrive/POSTAG/BERT_model/Roberta', args=model_args, use_cuda=torch.cuda.is_available(), labels=_POS_TAGS
)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_)

  0%|          | 0/11 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/657 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: C

Training loss,█▇▅▇▆▆▇█▃▄▆▃▄▄▄▄▂▂▂▃▂▄▂▃▃▂▂▃▃▃▄▃▂▂▃▂▁▄▂▂
eval_loss,███▇▇▆▅▅▅▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▂▂▂▂▂
f1_score,▁▁▁▂▂▃▄▄▄▆▆▆▆▆▇▇▇▇▇▇▇█▇██████▇██████████
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,▁▂▃▄▆▆████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆
precision,▁▁▁▂▂▃▄▄▄▆▆▆▆▆▇▇▇▇▇▇▇█▇████▇█▇██████████
recall,▁▁▁▂▂▃▄▄▄▅▆▆▆▆▇▇▇▇▇▇▇███████████████████
train_loss,▄▅▆▅▅█▅▇▄▅▃▃▃▃▃▅▂▃▂▃▃▄▄▃▃▁▂▃▂▂▁▂▂▁▂▂▁▁▂▂
Training loss,0.06736
eval_loss,0.16011
f1_score,0.93574


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wan

In [ ]:
with open('/content/pos_test.txt') as f:
  text = f.read()
text_split = text.split('\n')
word_list = [word if word != '' else '_' for word in text_split[:-2]]
print(len(word_list))
word_overlap = [' '.join(word_list[int(i*12): int(i*12+60)])for i in range(len(word_list)//12)]

predictions, raw_outputs = model.predict(word_overlap)
from collections import Counter
from tqdm.auto import tqdm
ans_counter = [Counter() for i in range(len(word_list))]
for i in tqdm(range(len(word_list)//12)):
  counts = ans_counter[int(i*12): int(i*12+60)]
  data = predictions[i]
  for each_counter, each_data in zip(counts, data):
    each_counter.update({list(each_data.values())[0]:1})
final = [max(counter, key=counter.get)for counter in ans_counter]


103913


  0%|          | 0/13 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1083 [00:00<?, ?it/s]

  0%|          | 0/8659 [00:00<?, ?it/s]

In [ ]:
submit = pd.read_csv('/content/pos_sample_submission.csv')
submit['Predicted'] = final
submit.to_csv('submission.csv',index = False)